In [1]:
from dataclasses import dataclass
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser

In [2]:
config = ConfigParser()
config.read("../.config")

['../.config']

In [3]:
API_KEY = config['GCP']['API_KEY']
radius = 1500  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [4]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None


latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")


In [5]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


Name: Luxembourg
Address: Luxembourg
Place ID: ChIJVyzznc1IlUcREG0F0dbRAAQ
---
Name: Hôtel Parc Belle-Vue
Address: 5 Avenue Marie-Thérèse, Luxembourg
Place ID: ChIJ3e9sj9VIlUcRUU6bnYZ2tcE
---
Name: Hotel Vauban
Address: 10 Place Guillaume II, Luxembourg
Place ID: ChIJQVrlNdNIlUcRa3v0FrkGQ-g
---
Name: French Café Hotel
Address: 14 Place d'Armes, Luxembourg
Place ID: ChIJz1niVCtPlUcROFfnez0PXz0
---
Name: Hotel Christophe Colomb
Address: 10 Rue d'Anvers, Luxembourg
Place ID: ChIJHeKzA9FIlUcR_UzTEdulOFs
---
Name: Youth Hostel
Address: 2 Rue du Fort Olisy, Luxembourg
Place ID: ChIJS5A_WTJPlUcRtAr15AXUGNs
---
Name: Novotel Luxembourg Centre
Address: 35 Rue du Laboratoire, Luxembourg
Place ID: ChIJl5q7iM5IlUcROv7uDbapuVo
---
Name: Grand Hotel Cravat
Address: 29 Boulevard Franklin Delano Roosevelt, Luxembourg
Place ID: ChIJi-jQXNNIlUcR77bXMeT_HSw
---
Name: Best Western Plus
Address: 3 Avenue Victor Hugo, Luxembourg
Place ID: ChIJld6G1ClPlUcRbXoEwx9_WUs
---
Name: Meliá Luxembourg
Address: 1 Par

In [7]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


Name: Hôtel Parc Belle-Vue
Address: 5 Av. Marie-Thérèse, 2132 Hollerich Luxembourg
Phone Number: 45 61 41 1
Website: https://www.goereshotels.com/belle-vue-en/


In [8]:
place_details['result']['place_id']

'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'

In [16]:
# Connect to DuckDB
con = duckdb.connect('places.db')

In [17]:
df = con.execute('SELECT * FROM places').fetchdf()

In [19]:
df

,place_id,name,address,latitude,longitude,website
0,ChIJVyzznc1IlUcREG0F0dbRAAQ,Luxembourg,Luxembourg,49.611621,6.131935,N/A
1,ChIJ3e9sj9VIlUcRUU6bnYZ2tcE,Hôtel Parc Belle-Vue,"5 Avenue Marie-Thérèse, Luxembourg",49.607684,6.122800,N/A
2,ChIJiVm9mNZFlUcR1lpH199XBOA,Hôtel Empire,"34 Place de la Gare, Luxembourg",49.599603,6.132795,N/A
3,ChIJ6VucqxJPlUcROccW2iDb5Bw,Parc Hotel Alvisse,"120 Route d'Echternach, Luxembourg",49.639720,6.154082,N/A
4,ChIJQVrlNdNIlUcRa3v0FrkGQ-g,Hotel Vauban,"10 Place Guillaume II, Luxembourg",49.610940,6.130641,N/A
...,...,...,...,...,...,...
115,ChIJSx2TDvL9v0cRcmZA9CXdmHE,Usträicherbetrieb Ludes s.à r.l.,"1 Am Walebroch, Dikrech",49.862527,6.145311,N/A
116,ChIJO8nKoZVVlUcRsxqxPkmUGuU,CONVIS s.c.,"4 Zone Artisanale Et Commerciale, Ettelbruck",49.839596,6.094361,N/A
117,ChIJH5DxzJJVlUcR4UcJRdtqeGo,Voyages Flammang,"34 Grand-Rue, Ettelbruck",49.846491,6.099198,N/A
118,ChIJNUfTgxhXlUcRtC9JDOkbWU0,Administration Communale de Nommern,"31 Rue Principale, Nommern",49.794824,6.173374,N/A


In [15]:
con.close()

In [5]:
def get_cities_towns(api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address=Luxembourg&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == 'OK':
        results = data['results']
        places = []
        for result in results:
            places.append(result['formatted_address'])
        return places
    else:
        print(f"Error: {data['status']}")

# Replace 'YOUR_API_KEY' with your actual Google Maps API key
cities_towns = get_cities_towns(API_KEY)
print(cities_towns)


['Luxembourg']
